## Pya Backend Examples
by TH, 2023-05-27

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import time
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8,2)
%matplotlib widget
import mesonic
import pya 
from pya import Asig, Aspec, Astft

In [ ]:
pya.device_info();

In [ ]:
# context.backend.quit() # pya.Aserver.default  # after quit() this will be closed

In [ ]:
from mesonic.backend.backend_pya import PyaSynthDef
context = mesonic.create_context("pya", device=4)
context.backend.aserver # Aserver.default

In [ ]:
context.managers

In [ ]:
context.timeline

In [ ]:
context.stop()

In [ ]:
context.reset()  # keep_canvas=True

In [ ]:
context.synths

pya synths are just functions to create an asig
- spawning in realtime is .play(onset=absolute_time) 
- spawning in nrt is acanvas.x[{time_from_start:None}] += synfn(parameters)
- pya_synths are immutable by default: the whole signal is computed 'one-shot'
- context from other synths or state at time is impossible to take on
- a stop message to a synths (at time t_stop) could only be faked by slicing the signal before adding to canvas

In [ ]:
def pya_syn_s1(out=0, freq=400, dur=0.3, amp=0.3):
    """pya sine tone synth"""
    return pya.Ugen().sine(freq=freq, amp=amp, dur=dur).fade_out(dur)

PyaSynthDef("s1", pya_syn_s1).add()


def pya_syn_s2(freq=440, amp=1.0, dur=0.1, width=1.0):
    """pya sawtooth tone synth"""
    return pya.Ugen().sawtooth(freq=freq, amp=amp, dur=dur, width=width)

PyaSynthDef("s2", pya_syn_s2).add()

In [ ]:
PyaSynthDef.synth_defs['s1'](freq=1000, amp=0.1).play()
PyaSynthDef.synth_defs['s2'](freq=500, amp=0.1).play(onset=0.5)

In [ ]:
PyaSynthDef.synth_defs

In [ ]:
context.timeline

In [ ]:
s1 = context.synths.create("s1", mutable=False)
context.synths

In [ ]:
context.timeline.plot()

In [ ]:
# create some test tones
context.reset()
for i, t in enumerate([0, 0.2, 0.4, 0.8]):
    with context.at(t):
        s1.start(freq=800+40*i, amp=0.1)
context.timeline.plot()

In [ ]:
# and/or create a discrete parameter mapping sonification
context.reset()
import numpy as np
for i, t in enumerate(np.random.rand(100)):
    with context.at(t*2):
        s1.start(freq=int(1000+np.random.random(1)[0]*800), dur=0.2, amp=0.05)
context.timeline.plot()

In [ ]:
# render to a fresh pya canvas
context._backend.clear_acanvas() # clear canvas
context.render() # render (file name irrelevant as rendered in memory

In [ ]:

acanvas = context.backend.acanvas
plt.figure(); acanvas.plot(lw=0.5).play(onset=0.3)

In [ ]:
context.render()
ac = context.backend.acanvas.play()
ac

## realtime pya synthesis

In [ ]:
context.stop()
context.reset()

In [ ]:
context.timeline

In [ ]:
pb = context.enable_realtime()
pb

In [ ]:
# find howto reduce latency via context.playback

In [ ]:
s1.start(freq=890, amp=0.3)

In [ ]:
# the context manager now yields the current time
# of the realtime Playback from the Context
with context.now() as timepoint:
    s1.start(dur=0.2, freq=440)
    print(f"Started s1 at {timepoint}")

In [ ]:
context.reset()
for i, t in enumerate([0, 0.2, 0.4, 0.8]):
    with context.at(t):
        s1.start(freq=800+40*i, amp=0.1)
context.timeline.plot()

In [ ]:
context.reset()
context.timeline

In [ ]:
# and/or create a discrete parameter mapping sonification
context.reset()
import numpy as np
for i, t in enumerate(np.random.rand(100)):
    with context.at(t*2):
        s1.start(freq=int(1000+np.random.random(1)[0]*800), dur=0.2, amp=0.05)
context.timeline.plot()

In [ ]:
from sc3nb import midicps

In [ ]:
context.reset()
pb.time = 0 
ofs = 0 # np.random.random()
print(ofs)
for i, t in enumerate(np.linspace(0, 0.5, 17, endpoint=True)):
    with context.at(t+ofs):
        s1.start(freq=midicps(90+i), amp=0.1)
context.timeline.plot()

In [ ]:
PyaSynthDef.get_description("s1")

In [ ]:
plt.figure()
PyaSynthDef.synth_defs['s1'](0, 480, 0.3, 1).plot().play(onset=0.1)

In [ ]:
pb = context.disable_realtime()

## Buffer

- BackendPya doesn't need buffers as anything is memory-mapped anyway.
- But for compatability with other backends, we implement Buffers as such also
- By definition, a Buffer is simply storing data in the Backend. 
- In case of BackendPya, the buffers are stored in the BackendPya attribute buffers as dictionary with 
  - keys being the bufnum, and 
  - values being the Asig that is the buffer storage.

Let's create a Buffer from a file.

In [ ]:
buf = context.buffers.from_file("files/clack.wav")
buf2 = context.buffers.from_file("files/da.wav")


In [ ]:
buf, buf2

In [ ]:
# the buffer is also stored in _buffers[buf] in the BufferManager
plt.figure(); context.buffers._buffers[buf].plot().stereo().play(onset=0.2, rate=0.1)

Using this Buffer we can create a Synth that will playback this Buffer.

In [ ]:
context.synths

In [ ]:
buf_synth = context.synths.from_buffer(buf)
buf_synth

In [ ]:
PyaSynthDef.synth_defs

In [ ]:
PyaSynthDef.synth_descs

In [ ]:
context.buffers._buffers[buf]

In [ ]:
PyaSynthDef.synth_defs['pya_playbuf_files/clack.wav_arrayindexed'](out=0, rate=1, amp=0.5).play(onset=0.2)

In [ ]:
buf

In [ ]:
# the pyabuffer is just an asig and can be found here:
context.buffers._buffers[buf]

In [ ]:
# create buffer synths in render
context.reset()
context.disable_realtime()
context._backend.clear_acanvas()

buf_synth = context.synths.from_buffer(buf)

t = 0.2
for k in range(100):
    t += 3*np.random.random() * 0.05
    with context.at(t):
        buf_synth.start(rate=0.5*np.random.random()+0.5, amp=0.2)

context.render("")
ac = context.backend.acanvas
ac.play()

And lets use the realtime mode so we can simply start 100 Buffer Synths

In [ ]:
context.reset()
context.enable_realtime()
context._backend.clear_acanvas()

buf_synth = context.synths.from_buffer(buf)

t = 0.1
for k in range(100):
    t += np.random.random() * 0.05
    with context.at(t):
        buf_synth.start(rate=0.5*np.random.random()+0.5, amp=0.2)

In [ ]:
context.time = 0

Notice that the resulting Synth is a mutable Synth.

In [ ]:
buf_synth.mutable

In [ ]:
context.stop()